### Preprocess Raw-data
start env preprocess


In [1]:
long_name=('B4 (665 nm)',
 'B3 (560 nm)',
 'B2 (490 nm)',
 'B8 (842 nm)',
 'SRB5 (705 nm)',
 'SRB6 (740 nm)',
 'SRB7 (783 nm)',
 'SRB8A (865 nm)',
 'SRB11 (1610 nm)',
 'SRB12 (2190 nm)') 
import os
# path to 'deepSat' containing: 
# "raw-data/AOI/tile_selection.gpkg", 
# 'raw-data/AOI/patch_size_features.gpkg', 
# 'raw-data/LULC-Sweden/*/Data/*.gpkg'
# 'raw-data/sentinel-2/Tidsperiod-x/'

root_path = '/home/exjobbare/projects/datadrive0/deepSat/'
timeperiod = 1 # timeperiod (tp) 1 eller 2

patch_size = 256  #choose between ['64','128','256']
source = 'dsen2' # for savedir... e.g. TCI => TCI_256 if patch_size=256
data_source = source + '_{}_PERC'.format(patch_size)
tp = 'timeperiod{}'.format(timeperiod)
sdir = os.path.join(root_path,'processed-data/{source}/{tp}'.format(source=data_source,tp=tp))

import glob
import h5netcdf
import xarray as xr
import numpy as np
import pandas as pd
import rioxarray as rio
from geopandas import read_file
from tqdm.notebook import tqdm
from preprocess.uaUtils import open_fua
from preprocess.s2Utils import search_product,open_tile,clip_tile, get_tile_info, get_prod_info
from preprocess.utils import rasterize,get_obj_within,str_to_oao, save_cube,class_dict,org_files

xr.set_options(keep_attrs=True)


# path to Urban Atlas (UA) data
ua_paths = glob.glob(os.path.join(root_path,'raw-data/LULC-Sweden/*/Data/*.gpkg'))

# path to s2 products
#'../dsen2_results/{}/'
s2_path = os.path.join(root_path,'../dsen2_results/{}/'.format(tp))
#s2_path = os.path.join(root_path,'raw-data/sentinel-2/{}/'.format(tp))
print('Looking for sentinel products in:\n',s2_path,'\n')

# get crs
ua_crs = read_file(ua_paths[0]).crs

# path to s2 tiles to use 
s2tiles = read_file(os.path.join(root_path,'raw-data/AOI/tile_selection.gpkg')).to_crs(ua_crs)

# path to scb patches of size*size
scb_grid = read_file(os.path.join(root_path,'raw-data/AOI/patch_size_features.gpkg'),layer = "patch_{}".format(patch_size)).to_crs(ua_crs)


# check that desired s2tiles existst in s2_path
print('---- Found sentinel products: ----')
for tile in s2tiles.itertuples():
    prod,date= search_product(s2_path,tile.Name,source='PERC')
    if prod:
        for i, prod in enumerate(prod):
            fn=os.path.basename(prod)
            print('Found:',get_prod_info(fn)['Tile'],pd.to_datetime(date[i]),fn)
    else:
        print('No found sentinel products for tile:',tile.Name)
          
# list found FUAS
print('\n---- Found LULC data: ----')
if ua_paths:
    for ua_path in ua_paths:
        print('Found:',os.path.basename(ua_path))
else:
    print('No LULC data found')
    

print('\n Save processed data to: \n',  sdir)

Looking for sentinel products in:
 /home/exjobbare/projects/datadrive0/deepSat/../dsen2_results/timeperiod1/ 

---- Found sentinel products: ----
Found: T33UUB 2018-07-26 10:20:19 S2B_MSIL2A_20180726T102019_N9999_R065_T33UUB_20220422T084315_PERC.tif
Found: T33VUC 2018-07-04 10:30:21 S2A_MSIL2A_20180704T103021_N9999_R108_T33VUC_20220421T224702_PERC.tif
Found: T33VUD 2018-06-29 10:30:19 S2B_MSIL2A_20180629T103019_N9999_R108_T33VUD_20220422T073547_PERC.tif
Found: T33VUE 2018-07-04 10:30:21 S2A_MSIL2A_20180704T103021_N9999_R108_T33VUE_20220421T230429_PERC.tif
Found: T33VVD 2018-07-26 10:20:19 S2B_MSIL2A_20180726T102019_N9999_R065_T33VVD_20220422T090148_PERC.tif
Found: T33VVE 2018-06-01 10:20:21 S2A_MSIL2A_20180601T102021_N9999_R065_T33VVE_20220421T205914_PERC.tif
Found: T33VVF 2018-07-04 10:30:21 S2A_MSIL2A_20180704T103021_N9999_R108_T33VVF_20220422T064154_PERC.tif
Found: T33VWE 2018-06-01 10:20:21 S2A_MSIL2A_20180601T102021_N9999_R065_T33VWE_20220421T211715_PERC.tif
Found: T33VWF 2018-06-

In [6]:
# sequential all fuas (original)
bad_patch=[]
for path in tqdm(ua_paths, desc='Total progress'):
    
    fua_labls,fua_bound = open_fua(path)             # open fua layers
    tiles_inters = s2tiles.sjoin(fua_bound, predicate='intersects')  #get intersecting tiles for fua
    fua_name = str_to_oao(fua_bound.fua_name[0])
    
    for tile in tqdm(tiles_inters.itertuples(),desc='Intersecting tiles',total=len(tiles_inters)): # for every intersecting tile
        
        # select patches within curr tile
        curr_tile = s2tiles[s2tiles.Name.isin([tile.Name])]                                               
        patches_within = get_obj_within(get_obj_within(scb_grid,curr_tile),fua_bound) #select patches(grid) within curr_tile and within fua
        #savedir
        savedir = os.path.join(sdir,'{}/{}/'.format(fua_name,tile.Name))
        #print(savedir)
        #open curr tile 
        s2_tile = open_tile(tile.Name,s2_path,source='PERC')
    
        for patch in tqdm(patches_within.itertuples(),total=len(patches_within), desc='FUA: {}, Tile: {}'.format(fua_name,tile.Name)):
            
            # get current patch and clip labels to patch extent
            curr_patch = scb_grid[(scb_grid.id.isin([patch.id]))]
            # clip tile and labels to patch extent
            s2_patch = clip_tile(s2_tile,curr_patch)
            s2_patch = s2_patch.astype(np.float32)
            patch_labls = fua_labls.clip(curr_patch)
            #rasterize patch labels 
            try:
                cube = rasterize(patch_labls)
            except:
                bad_patch.append(patch.id)
            else:    
                #merge s2_patch and patch labels
                cube = cube.merge(s2_patch.rio.reproject_match(cube)) #append to cube
                # reduce coordinate dimensions by one
                if cube.dims['x'] > patch_size:
                   cube=cube.isel(x=slice(None, -1), y=slice(None, -1))           
            
                #set attributes to cube before saving
                cube.attrs["patch_id"] = patch.id
                cube.attrs['FUA'] = fua_name
                cube.attrs['long_name']=long_name
                for key,value in get_tile_info(tile.Name,s2_path).items():
                    cube.attrs[key] = value
                #cube.train_id.attrs['_fillValue']=255
                #cube.class_code.attrs['_fillValue']=255
                save_cube(cube,savedir)

Total progress:   0%|          | 0/12 [00:00<?, ?it/s]

Intersecting tiles:   0%|          | 0/4 [00:00<?, ?it/s]

FUA: STOCKHOLM, Tile: 33VXF:   0%|          | 0/324 [00:00<?, ?it/s]

FUA: STOCKHOLM, Tile: 33VXG:   0%|          | 0/233 [00:00<?, ?it/s]

FUA: STOCKHOLM, Tile: 34VCL:   0%|          | 0/284 [00:00<?, ?it/s]

...

FUA: VASTERAS, Tile: 33VXG:   0%|          | 0/3 [00:00<?, ?it/s]

Intersecting tiles:   0%|          | 0/3 [00:00<?, ?it/s]

FUA: NORRKOPING, Tile: 33VWE:   0%|          | 0/81 [00:00<?, ?it/s]

FUA: NORRKOPING, Tile: 33VWF:   0%|          | 0/47 [00:00<?, ?it/s]

FUA: NORRKOPING, Tile: 33VXF: 0it [00:00, ?it/s]

In [13]:
# calculate min max to check for bad data

from tqdm.notebook import tqdm
from dataset.datasets import s2stats, sentinel
#import torch
import numpy as np
import os

dset= s2stats(root_dir='processed-data/dsen_2_256_new/*/*/')
max_vals=np.zeros((len(dset),10))
min_vals=np.zeros((len(dset),10))

count = 0
paths = []

for idx, (img,_,path) in enumerate(dset):
   
    maxs = img.numpy().max(axis=(1,2))
    mins = img.numpy().min(axis=(1,2))
    
    if any(maxs >3.4e+38):
        count +=1
        paths.append(path)
    
    max_vals[idx,:]=img.numpy().max(axis=(1,2))
    min_vals[idx,:]=img.numpy().min(axis=(1,2))
    
    
maxs= max_vals.max(axis=0)
mins= min_vals.min(axis=0)

print(maxs)
print(mins)
print(count)


#for path in paths: 
#    os.remove(path)

ModuleNotFoundError: No module named 'torch'

In [19]:
sdir

'/home/exjobbare/projects/datadrive0/deepSat/processed-data/raw_256_PERC/timeperiod1'

### split train/test/val

1. split dataset (if satisfied, execute step 2)
2. reorganize files after split

In [21]:
#1 split dataset
train = 0.8
test = 0.5 #of remaining -train
sdir='/home/exjobbare/projects/datadrive0/deepSat/processed-data/dsen_2_256_new/timeperiod1/'
df = np.asarray(glob.glob(os.path.join(sdir,'*/*/*.nc')))
total = len(df)
msk = np.random.rand(len(df)) <train
len(msk[msk==True])/len(msk)

train = df[msk]
testval = df[~msk]
msk = np.random.rand(len(testval))<test
test=testval[msk]
val = testval[~msk]

print('total:',total,'train:', len(train) ,round(len(train)/len(df)*100,0),'test:',len(test),round(len(test)/len(df)*100,0),'val:',len(val),round(len(val)/len(df)*100,0))

total: 3168 train: 2524 80.0 test: 315 10.0 val: 329 10.0


In [22]:
# step 2 copy files to 'source_split' '/train' '/test' or '/val'
if train.size>0: 
    org_files(train,mode='train')
if test.size>0:
    org_files(test,mode='test')
if val.size>0:
    org_files(val,mode='val')


/home/exjobbare/projects/datadrive0/deepSat/processed-data/dsen_2_256_new_split/timeperiod1/train
/home/exjobbare/projects/datadrive0/deepSat/processed-data/dsen_2_256_new_split/timeperiod1/test
/home/exjobbare/projects/datadrive0/deepSat/processed-data/dsen_2_256_new_split/timeperiod1/val


In [14]:
sdir
       
    

'/home/exjobbare/projects/datadrive0/deepSat/processed-data/raw_256_PERC/timeperiod1'

In [24]:
# preprocess fua one by one 
# run in terminal: ipcluster start -n N  where -n is number of clusters, 
# the parallel computing does not work properly (it runs sequentialy on each cluster so this needs further investigation)
# uncomment the one to process (rc[i] where i defines wich cluster to use, make sure i<N)

from ipyparallel import Client


def process_fua(path):


    bad_patch=[]
    fua_labls,fua_bound = open_fua(path)             # open fua layers
    tiles_inters = s2tiles.sjoin(fua_bound, predicate='intersects')  #get intersecting tiles for fua
    fua_name = str_to_oao(fua_bound.fua_name[0])
    
    for tile in tqdm(tiles_inters.itertuples(),desc='Intersecting tiles',total=len(tiles_inters)): # for every intersecting tile
        
        # select patches within curr tile
        curr_tile = s2tiles[s2tiles.Name.isin([tile.Name])]                                               
        patches_within = get_obj_within(get_obj_within(scb_grid,curr_tile),fua_bound) #select patches(grid) within curr_tile and within fua
        #savedir
        savedir = os.path.join(sdir,'{}/{}/'.format(fua_name,tile.Name))
        #print(savedir)
        #open curr tile 
        s2_tile = open_tile(tile.Name,s2_path)
        
        for patch in tqdm(patches_within.itertuples(),total=len(patches_within), desc='FUA: {}, Tile: {}'.format(fua_name,tile.Name)):
            
            # get current patch and clip labels to patch extent
            curr_patch = scb_grid[(scb_grid.id.isin([patch.id]))]
            # clip tile and labels to patch extent
            s2_patch = clip_tile(s2_tile,curr_patch)
            patch_labls = fua_labls.clip(curr_patch)
            #rasterize patch labels 
            try:
                cube = rasterize(patch_labls)
            except:
                bad_patch.append(patch.id)
            else:    
                #merge s2_patch and patch labels
                cube = cube.merge(s2_patch.rio.reproject_match(cube)) #append to cube
                # reduce coordinate dimensions by one
                if cube.dims['x'] > patch_size:
                   cube=cube.isel(x=slice(None, -1), y=slice(None, -1))           
            
                #set attributes to cube before saving
                cube.attrs["patch_id"] = patch.id
                cube.attrs['FUA'] = fua_name
                for key,value in get_tile_info(tile.Name,s2_path).items():
                    cube.attrs[key] = value


                cube.train_id.attrs['_fillValue']=255
                cube.class_code.attrs['_fillValue']=255
                save_cube(cube,savedir)
           

    return(savedir)

rc = Client()
#rc= cluster
#dwiev = rc[:]


#process_fua(ua_paths[0])    
#stockholm=rc[0].apply(process_fua(ua_paths[0])) #Stockholm
#jonkoping=rc[1].apply(process_fua(ua_paths[1]))  #JONKOPING
#linkoping=rc[2].apply(process_fua(ua_paths[2]))  #Linköping (33VWF not processed)
#malmo=rc[3].apply(process_fua(ua_paths[3]))  # Malmö  
#goteborg=rc[4].apply(process_fua(ua_paths[4])) #Gotebprg
#umea=rc[5].apply(process_fua(ua_paths[5]))      #umea
#uppsala=rc[6].apply(process_fua(ua_paths[6])) #uppsala
#orebro=rc[7].apply(process_fua(ua_paths[7]))    #orebro
#boras=rc[8].apply(process_fua(ua_paths[8]))    #boras
#helsingborg=rc[9].apply(process_fua(ua_paths[9]))   #helsingborg
#vasteras=rc[10].apply(process_fua(ua_paths[10])) #vasteras
#norrkoping= rc[11].apply(process_fua(ua_paths[11])) #norrkoping


In [36]:
##experimental remove later 


#print(result)

import ipyparallel as ipp
cluster = ipp.Cluster(n=6)
cluster.start_cluster_sync()

rc = cluster.connect_client_sync()
rc.wait_for_engines(6)
rc.ids
dview = rc[:]

with dview.sync_imports():
    from preprocess.uaUtils import open_fua
    import os
    import glob
    import h5netcdf
    import rioxarray
    import geopandas
    import numpy
    from tqdm.notebook import tqdm
    from preprocess.uaUtils import open_fua
    from preprocess.s2Utils import search_product,open_tile, get_tile_info, get_prod_info
    from preprocess.utils import str_to_oao, class_dict,org_files
    
    import geopandas as gp
    import os

    from geocube.api.core import make_geocube
    from functools import partial
    from geocube.rasterize import rasterize_image
    from preprocess.classDict import class_dict
    import shutil
    import numpy
    import glob
    import geopandas


    from geocube.api.core import make_geocube
    from functools import partial
    from geocube.rasterize import rasterize_image
    from preprocess.classDict import class_dict
    import shutil
    import numpy
    import glob
    

    
@dview.parallel(block=True)    
def rem_rasterize(patch_labls):
    patch_labls['train_id'] = patch_labls['code_2018'].apply(lambda x: numpy.uint16(class_dict[str(x)]['train_id']))
    patch_labls['class_code'] = patch_labls['code_2018'].apply(lambda  x:  numpy.uint16(int(x)))
    roads = patch_labls[patch_labls.code_2018.isin(['12210','12220','12230'])]
    
    cube  = make_geocube(
        vector_data=patch_labls,
        measurements=['class_code','train_id'],
        resolution=(10,-10),
        rasterize_function=partial(rasterize_image,dtype=numpy.dtype('uint16')),
        fill = 0
    )
    
    roads = make_geocube(
        vector_data=roads,
        measurements=['class_code','train_id'],
        resolution=(10,-10),
        rasterize_function=partial(rasterize_image,all_touched=True,dtype=numpy.dtype('uint16')),
            fill=0
    )
    
    cube = roads.where(roads.train_id!=0).combine_first(cube)  #merge roads  
    cube = cube.assign(train_id = lambda ds: ds.train_id.astype('uint16'))
    cube = cube.assign(class_code = lambda ds: ds.class_code.astype('uint16'))
    return(cube)

@dview.parallel(block=True)
def rem_save_cube(cube,savedir):
                # patch filename and 
    filename = "PATCH_{i_d}_{tile}_{start}_{fua}_{size}.nc".format(i_d=cube.attrs['patch_id'],
                                                                   tile=cube.attrs['Tile'],
                                                                   start=cube.attrs['sensing_start'],
                                                                   fua=cube.attrs['FUA'],
                                                                   size=cube.dims['x'])
              
    if not os.path.exists(savedir):
        os.makedirs(savedir)
    cube.to_netcdf(path=os.path.join(savedir,filename),format='NETCDF4')


    
#@dview.remote(block=True)
#def proc_fua(path,s2tiles,scb_grid,sdir,s2_path):
def proc_fua(path):
        
    def clip_tile(tile,clip_shape):
        bounds  = clip_shape.to_crs(tile.rio.crs).total_bounds
        patch = tile.rio.clip_box(minx=bounds[0],
                                  miny=bounds[1],
                                  maxx=bounds[2],
                                  maxy=bounds[3])
        return(patch)

    def get_obj_within(obj,roi):
        return obj[(obj.id.isin(obj.sjoin(roi,predicate='within').id))]

    res= None
    #sdir = sdir[0]
    #s2_path=s2_path[0]
    #scb_grid = scb_grid[0]
    #path = path[0]
    #s2tiles = s2tiles[0]
    bad_patch=[]
    fua_labls,fua_bound = open_fua(path)             # open fua layers
    tiles_inters = s2tiles.sjoin(fua_bound, predicate='intersects')  #get intersecting tiles for fua
    fua_name = str_to_oao(fua_bound.fua_name[0])
    
    for tile in tqdm(tiles_inters.itertuples(),desc='Intersecting tiles',total=len(tiles_inters)): # for every intersecting tile
        
        # select patches within curr tile
        curr_tile = s2tiles[s2tiles.Name.isin([tile.Name])]                                               
        patches_within = get_obj_within(get_obj_within(scb_grid,curr_tile),fua_bound) #select patches(grid) within curr_tile and within fua
        #savedir
        savedir = os.path.join(sdir,'{}/{}/'.format(fua_name,tile.Name))
        #print(savedir)
        #open curr tile 
        s2_tile = open_tile(tile.Name,s2_path)
        
        for patch in tqdm(patches_within.itertuples(),total=len(patches_within), desc='FUA: {}, Tile: {}'.format(fua_name,tile.Name)):
            
            # get current patch and clip labels to patch extent
            curr_patch = scb_grid[(scb_grid.id.isin([patch.id]))]
            # clip tile and labels to patch extent
            s2_patch = clip_tile(s2_tile,curr_patch)
            patch_labls = fua_labls.clip(curr_patch)
            #rasterize patch labels 
            try:
                cube = rem_rasterize(patch_labls)
            except:
                bad_patch.append(patch.id)
            else:    
                #merge s2_patch and patch labels
                
                cube = cube.merge(s2_patch.rio.reproject_match(cube)) #append to cube
                # reduce coordinate dimensions by one
                if cube.dims['x'] > patch_size:
                    cube=cube.isel(x=slice(None, -1), y=slice(None, -1))           
            
                #set attributes to cube before saving
                cube.attrs["patch_id"] = patch.id
                cube.attrs['FUA'] = fua_name
                for key,value in get_tile_info(tile.Name,s2_path).items():
                    cube.attrs[key] = value


                cube.train_id.attrs['_fillValue']=255
                cube.class_code.attrs['_fillValue']=255
                rem_save_cube(cube,savedir)
           
    
    return('test')
#scb_grid[scb_grid.id.isin(scb_grid.sjoin(curr_tile,predicate='within').id]


#print(proc_fua([ua_paths[3]],[s2tiles],[scb_grid],[sdir],[s2_path]))
print(proc_fua(ua_paths[0]))

Starting 6 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/6 [00:00<?, ?engine/s]

importing open_fua from preprocess.uaUtils on engine(s)
importing os on engine(s)
importing glob on engine(s)
importing h5netcdf on engine(s)
importing rioxarray on engine(s)
importing geopandas on engine(s)
importing numpy on engine(s)
importing tqdm from tqdm.notebook on engine(s)
importing search_product,open_tile,get_tile_info,get_prod_info from preprocess.s2Utils on engine(s)
importing str_to_oao,class_dict,org_files from preprocess.utils on engine(s)
importing make_geocube from geocube.api.core on engine(s)
importing partial from functools on engine(s)
importing rasterize_image from geocube.rasterize on engine(s)
importing class_dict from preprocess.classDict on engine(s)
importing shutil on engine(s)


Intersecting tiles:   0%|          | 0/4 [00:00<?, ?it/s]

FUA: STOCKHOLM, Tile: 33VXF:   0%|          | 0/324 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### Compact Naming Convention

The compact naming convention is arranged as follows:

MMM_MSIXXX_YYYYMMDDHHMMSS_Nxxyy_ROOO_Txxxxx_<Product Discriminator>.SAFE

The products contain two dates.

The first date (YYYYMMDDHHMMSS) is the datatake sensing time.
The second date is the "Product Discriminator" field, which is 15 characters in length, and is used to distinguish between different end user products from the same datatake. Depending on the instance, the time in this field can be earlier or slightly later than the datatake sensing time.

The other components of the filename are:

* MMM: is the mission ID(S2A/S2B)
* MSIXXX: MSIL1C denotes the Level-1C product level/ MSIL2A denotes the Level-2A product level
* YYYYMMDDHHMMSS: the datatake sensing start time
* Nxxyy: the PDGS Processing Baseline number (e.g. N0204)
* ROOO: Relative Orbit number (R001 - R143)
* Txxxxx: Tile Number field

SAFE: Product Format (Standard Archive Format for Europe)

Source https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/naming-convention

In [ ]:
bad_p = ['46008',
 '48950',
 '47261',
 '45171',
 '45997',
 '45592',
 '47692',
 '41799',
 '48950',
 '48514',
 '48514',
 '60278',
 '60679',
 '56922',
 '53536',
 '56077',
 '58600',
 '59854',
 '59009',
 '56053',
 '51452',
 '9139',
 '8715',
 '7450',
 '4934',
 '5354',
 '4082',
 '5349',
 '4923',
 '11652',
 '8299',
 '9551',
 '10387',
 '8719',
 '7050',
 '9139',
 '8719',
 '7470',
 '8299',
 '60303',
 '64920',
 '56945',
 '62823',
 '59454',
 '59460',
 '57356',
 '65340',
 '60731',
 '60723',
 '61151',
 '59457',
 '60311',
 '54844',
 '59891',
 '62403',
 '61143',
 '59880',
 '59454',
 '59871',
 '58607',
 '63229',
 '63226',
 '57346',
 '60288',
 '63640',
 '62809',
 '59024',
 '16274',
 '15017',
 '15430',
 '13760',
 '15017',
 '45635',
 '42276',
 '42276',
 '47310',
 '45635',
 '15505',
 '16766',
 '15087',
 '26785',
 '26365',
 '28035',
 '28455',
 '28867',
 '28455',
 '28035',
 '33010',
 '31752',
 '37211',
 '35105',
 '34262',
 '38048',
 '41027',
 '40601',
 '43553',
 '41035',
 '38923',
 '35984',
 '40604',
 '39349',
 '18468',
 '19313',
 '20985',
 '18041',
 '17213',
 '17200',
 '19306',
 '19730',
 '18896']
